In [1]:
# mou give me 32 thyroid annotations to train a model, which labeled by xiaobai's tool
# I will use python to parse to labelme's syntax.
#from 21/08/21
# 22/01/29 thyroid nodule's modle format valid.

In [112]:
f1=r"/mnt/d/workspace/03-annotations/1.3.6.1.4.1.52026.66519049.5308.54-v1/1.3.6.1.4.1.52026.66519049.5308.55.1.1.json"

In [113]:
import json
with open(f1, 'r') as f:
    #data=f.read()
    j1 = json.load(f)
    print(len(j1))
    for i in j1:
        print(i)

5
PatientID
StudyID
data
series_instance_uid
sourceVIDEOFilePath


In [115]:
all_imgs_datas = j1['data']
print(len(all_imgs_datas), type(all_imgs_datas))
print(type(all_imgs_datas[0]))
j_data_items = all_imgs_datas[0]
for i in j_data_items:
    print(i)
print(j_data_items['index'])

245 <class 'list'>
<class 'dict'>
fileName
index
lesionCount
numberLayers
object
polygonCount
seriesNumber
size
sopUid
0


In [116]:
uids=[]
for data in all_imgs_datas:
    uids.append(int(data['sopUid'].split('.')[-1]))
uids.sort()
print(uids)

[0, 1, 2, 3, 4, 5, 6, 7, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235,

In [117]:
# get the absent annotations index
j = 0
for i in range(len(uids)):
    if i != uids[j]:
        print(i)
        #j=j-1
    else:
        j =j+1

8
9
10
11
12
13
14
15
16
34
35
36
37
38
39
40
41
42


In [5]:
j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

In [6]:
j_target_o = json.loads(j_target_s)

In [7]:
print(j_target_o)

{'shapes': [{'label': '颈动脉:CA', 'line_color': None, 'fill_color': None, 'points': [[296, 253], [295, 373], [430, 363], [427, 255]]}], 'lineColor': [0, 255, 0, 128], 'fillColor': [255, 0, 0, 128], 'imagePath': 'frm-0001.png', 'imageData': None}


In [8]:
import numpy
one_img = all_imgs_datas[0]
for item in one_img:
    print(item, type(item))
one_img_object = one_img['object']
print(type(one_img_object), len(one_img_object))

for img_data in all_imgs_datas:
    img_objs = img_data['object']
    n = int(img_data['sopUid'].split('.')[-1])
    print(f"img({n}) has {len(img_objs)} object.")
    polygonPoints = img_objs[0]['polygonPoint']
    print(f"polygonPoint length={len(polygonPoints)}")
    filename = f"frm-{(n+1):04d}"
    points_len = int((len(polygonPoints)))
    
    all_points = numpy.zeros((points_len, 2))
    all_points = all_points.tolist()
    print(len(all_points))
    for item in polygonPoints.items():
        #print("point item=", item, "len=", len(item))
        point_idx = int(item[0][1:])
        x0y1 = item[0][0]
        if 'x' == x0y1.lower():
            x0y1 = 0
        else:
            x0y1=1
        #print(f"point idx={point_idx}, coordinate={x0y1}")
        all_points[point_idx][x0y1] = int(item[1])
        #print(f">>>>>point[{point_idx}]=",all_points[int(point_idx)], ">>>>", all_points[point_idx][x0y1])
        #print(f"#all points={all_points[:20][:]}")
        #if (point_idx>10):
        #    break
    
    #print(f"all_points len={len(all_points)}")
    available_pts = []
    for pt in range(points_len):
        #print(f"point [{pt}]={all_points[pt]}")
        if all_points[pt][0] is None or all_points[pt][0]<10:
            #print(f"point[{pt}] is {all_points[pt]} delete it...")
            continue
        else:
            available_pts.append(all_points[pt])
            
    j_target_o['imagePath']=f"{filename}.png"
    j_target_o['shapes'][0]['label']='Thyroid'
    j_target_o['shapes'][0]['points']=available_pts
    
    with open(f"{filename}.json", 'w') as jfp:
        json.dump(j_target_o, jfp)
    
    #break

NameError: name 'all_imgs_datas' is not defined

In [122]:
!pwd

/mnt/d/srcs/jupyter_notebook


In [2]:
# the full function to generate labelme json from xiaobai's json
import json 
import os

j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

def convert_to_labelme_format(xb_json_file, param):
    """
        deal with one xiaobai's json which have multi-labelme json
    """
    if not os.path.isfile(xb_json_file):
        print(f"ERR: json file:{xb_json_file} not valid")
        return -1
    with open(xb_json_file, 'r') as jfp:
        jdata = json.load(jfp)
    all_imgs_datas = jdata['data']
    series_instance_uid = jdata['series_instance_uid']
    #--01 create home directory
    target_json_home = xb_json_file.split('.json')[0] + '.dcm_frms'
    target_json_home = target_json_home.replace("03-annotations", "01-videos")
    os.makedirs(target_json_home, exist_ok=True)
    print(f"target_json_home={target_json_home}")
    #--02 extract json per image 
    for img_data in all_imgs_datas:
        img_objs = img_data['object']
        n = int(img_data['sopUid'].split('.')[-1])
        print(f"img({n}) has {len(img_objs)} object.")
        polygonPoints = img_objs[0]['polygonPoint']
        print(f"polygonPoint length={len(polygonPoints)}")
        filename = f"frm-{(n+1):04d}"
        points_len = int((len(polygonPoints)))

        all_points = numpy.zeros((points_len, 2))
        all_points = all_points.tolist()
        for item in polygonPoints.items():
            point_idx = int(item[0][1:])
            x0y1 = item[0][0]
            if 'x' == x0y1.lower():
                x0y1 = 0
            else:
                x0y1=1
            all_points[point_idx][x0y1] = int(item[1])
        #--03 delete zeros
        available_pts = []
        for pt in range(points_len):
            #print(f"point [{pt}]={all_points[pt]}")
            if all_points[pt][0] is None or all_points[pt][0]<10:
                #print(f"point[{pt}] is {all_points[pt]} delete it...")
                continue
            else:
                available_pts.append(all_points[pt])
        #--04 create target labelme format json
        j_target_o['imagePath']=f"{filename}.png"
        j_target_o['shapes'][0]['label']='Thyroid'
        j_target_o['shapes'][0]['points']=available_pts
        #--05 write to sidk
        target_json_name = os.path.join(target_json_home, f"{filename}.json")
        with open(target_json_name, 'w') as jfp:
            json.dump(j_target_o, jfp)
    
    

def convert_all_xiaobai_jsons(json_root_dir, count=0):
    if not os.path.exists(json_root_dir):
        print(f"Dir not exist:{json_root_dir}")
        return -1
    subdirs = []
    dirs = sorted(os.listdir(json_root_dir))
    for item in dirs:
        path = os.path.join(json_root_dir, item)
        if os.path.isdir(path):
            subdirs.append(path)
    #print(f"subdirs={subdirs}")
    
    folder_details = []
    json_f_count = 0
    for ct, json_dir in enumerate(subdirs):
        print("*"*32)
        print('Handle with {0} [{1}/]'.format(json_dir, ct + 1))
        files_in_dir = os.listdir(json_dir)
        if len(files_in_dir) < 2:
            getLogger().info(f"Empty dirctory : {json_dir}")
            continue
        else:
            for json_file in files_in_dir:
                fullfilename = os.path.join(json_dir, json_file)
                if not os.path.isfile(fullfilename):
                    continue
                #print("file in dir:", fullfilename)
                convert_to_labelme_format(fullfilename, 100)
                json_f_count = json_f_count+1
                if count != 0 and count == json_f_count:
                    print(f"\n<<<already extract {count} directory json file~")
                    return
    return 0

In [3]:
##worked at 21/08/22
#jhomes=r"/mnt/d/workspace/03-annotations/"
#convert_all_xiaobai_jsons(jhomes, 0)

In [26]:
##working at 22/01/29
json_file=r"/mnt/d/workspace/220128-thyroid-nodules/220128-nodules/1.3.6.1.4.1.52026.66519049.5308.126.1.1.20220128001204810.json"

import json
import os

j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""


def convert_nodule_json(json_file):
    # 01-read from nodule-json-file
    with open(json_file, 'r') as f:
        json_ = json.load(f)
        print("json has item number=", len(json_))
        for i in json_:
            print(f"\titem[{i}]")
        all_imgs_datas = json_['data']
        print("total has data number=", len(all_imgs_datas), type(all_imgs_datas))
        print(type(all_imgs_datas[0]))
        one_data_item = all_imgs_datas[0]
        print(f"\titems in \\data\\")
        for i in one_data_item:
            print(f"\t\t{i}")
        print(f"\t\t lesions in data.")
        print("\t\t one_data_item['frameNumber']:", one_data_item['frameNumber'])
        for i in one_data_item["lesions"][0]:
            print(f"\t\t\t items in lesions:{i}, {type(i)}")
        lesions = one_data_item["lesions"]
        if(len(lesions)>0):
            polygonPoints = lesions[0]["polygonPoint"]
            print(f"polygonPoint:{polygonPoints}")
        if (len(lesions)>1):
            print("!"*20, "more than one lesion found.")
            
        #--02 extract json per image 
        for one_data_item in all_imgs_datas:
            frameNumber = one_data_item['frameNumber']
            lesions = one_data_item["lesions"]
            if(len(lesions)>0):
                polygonPoints = lesions[0]["polygonPoint"]
                #print(f"polygonPoint:{polygonPoints}")
            else:
                polygonPoints=[]
            if (len(lesions)>1):
                print("!"*20, "more than one lesion found.")
                print(f"polygonPoint length={len(polygonPoints)}")
            filename = f"frm-{(frameNumber):04d}"
            points_len = int((len(polygonPoints)))

            all_points = numpy.zeros((points_len, 2))
            all_points = polygonPoints

            #--03 delete zeros
            available_pts = []
            for pt in range(points_len):
                #print(f"point [{pt}]={all_points[pt]}")
                if all_points[pt][0] is None or all_points[pt][0]<10:
                    #print(f"point[{pt}] is {all_points[pt]} delete it...")
                    continue
                else:
                    available_pts.append(all_points[pt])
            #--04 create target labelme format json
            j_target_o['imagePath']=f"{filename}.png"
            j_target_o['shapes'][0]['label']='ThyNodu'
            j_target_o['shapes'][0]['points']=available_pts
            #--05 write to disk
            target_json_name = os.path.join(r"/mnt/d/workspace/220128-thyroid-nodules/220128-nodules/23.dcm_frms", f"{filename}.json")
            with open(target_json_name, 'w') as jfp:
                json.dump(j_target_o, jfp)
  
convert_nodule_json(json_file)

json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 112 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 lesions in data.
		 one_data_item['frameNumber']: 230
			 items in lesions:attributes, <class 'str'>
			 items in lesions:lesionID, <class 'str'>
			 items in lesions:markBoxType, <class 'str'>
			 items in lesions:pointCount, <class 'str'>
			 items in lesions:polygonPoint, <class 'str'>
polygonPoint:[[577.3333333333333, 369.3333333333333], [577.3333333333333, 368], [636, 360], [672, 372], [678.6666666666666, 414.66666666666663], [630.6666666666666, 428], [585.3333333333333, 428], [565.3333333333333, 400]]
